In [2]:
from pprint import pprint
from jnpr.junos import Device
from datetime import datetime
from jnpr.junos import exception as EzErrors
from lxml import etree
import os



In [3]:
from jnpr.junos.utils.config import Config

dev = Device('core1', user='lab', password='lab123', timeout = 10)
dev.open()

# data = dev.rpc.get_config(filter_xml="system/services", options={'format':'xml'})
data = dev.rpc.get_config(filter_xml="system/services", options={'format':'text'})
print(etree.tostring(data, encoding='unicode'))



<configuration-text>
## Last changed: 2017-11-14 09:53:29 HKT
system {
    services {
        ssh;
        netconf {
            ssh;
            traceoptions {
                file test.log;
            }
        }
    }
}
</configuration-text>



In [9]:
with Config(dev, mode='private') as cu:  
    cu.load('set system services netconf traceoptions file test.log', format='set')
    cu.pdiff()
    cu.commit()


[edit system services netconf]
+     traceoptions {
+         file test.log;
+     }



with Config(dev, mode='private') as cu:  
    cu.load('set interfaces ge-0/0/0 unit 0 family inet address 1.1.1.100/24', format='set', 
            merge=True)
    cu.pdiff()
    cu.commit()

In [20]:
%more ge000.conf
'''
interfaces {
replace:
    ge-0/0/0 {
        unit 0 {
            family inet {
                address 222.2.2.2/24;
            }
            family mpls;
        }
    }
}
'''



'\ninterfaces {\nreplace:\n    ge-0/0/0 {\n        unit 0 {\n            family inet {\n                address 192.168.12.1/24;\n            }\n            family mpls;\n        }\n    }\n}\n'

In [32]:
conf_file = "./ge000.conf"

with Config(dev, mode='private') as cu:  
#     cu.load('set interfaces ge-0/0/0 unit 0 family inet address 222.0.0.1/24', format='set', 
#             merge=True)
    cu.load(path=conf_file, merge=True)
    cu.pdiff()
    cu.commit()



[edit interfaces ge-0/0/0 unit 0 family inet]
        address 192.168.12.1/24 { ... }
+       address 222.2.2.2/24;



In [31]:
with Config(dev, mode='private') as cu:  
#     cu.load('set interfaces ge-0/0/0 unit 0 family inet address 222.0.0.1/24', format='set', 
#             merge=True)
    cu.load(path=conf_file)
    cu.pdiff()
    cu.commit()

None


In [28]:
conf_file = "./ge000_original.conf"

with Config(dev, mode='private') as cu:  
#     cu.load('set interfaces ge-0/0/0 unit 0 family inet address 222.0.0.1/24', format='set', 
#             merge=True)
    cu.load(path=conf_file, merge=True)
    cu.pdiff()
    cu.commit()

None


In [33]:
conf_file = "./ge000_original.conf"

with Config(dev, mode='private') as cu:  
#     cu.load('set interfaces ge-0/0/0 unit 0 family inet address 222.0.0.1/24', format='set', 
#             merge=True)
    cu.load(path=conf_file)
    cu.pdiff()
    cu.commit(comment='Configuring ge-0/0/0 interface')


[edit interfaces ge-0/0/0 unit 0 family inet]
-       address 222.2.2.2/24;



In [36]:
with Config(dev, mode='private') as cu:
    diff = cu.diff(rb_id=1)
    print(diff)



[edit interfaces ge-0/0/0 unit 0 family inet]
-       address 222.2.2.2/24;



In [ ]:
dev.close()

In [37]:
# Putting things all together
from jnpr.junos.exception import ConnectError
from jnpr.junos.exception import LockError
from jnpr.junos.exception import RpcError
from jnpr.junos.exception import CommitError
from jnpr.junos.exception import UnlockError

In [39]:
dev = Device('core1', user='lab', password='lab123', timeout = 10)

# open a connection with the device and start a NETCONF session
try:
    dev.open()
except ConnectError as err:
    print ("Cannot connect to device: {0}".format(err))


In [40]:
# Set up config object
cu = Config(dev)

In [42]:
# Lock the configuration
print ("Locking the configuration")
try:
    cu.lock()
except LockError as err:
    print ("Unable to lock configuration: {0}".format(err))
    dev.close()


Locking the configuration


In [47]:
rback = 1 
try:
    diff = cu.diff(rb_id=rback)
    print(diff)
    print("Rolling back the configuration by %d commit" % rback)
    cu.rollback(rb_id=1)
    print ("Committing the configuration")
    cu.commit()
except CommitError as err:
    print ("Error: Unable to commit configuration: {0}".format(err))
except RpcError as err:
    print ("Unable to rollback configuration changes: {0}".format(err))


[edit interfaces ge-0/0/0 unit 0 family inet]
-       address 222.2.2.2/24;

Rolling back the configuration by 1 commit
Committing the configuration


In [ ]:
finally:
    print ("Unlocking the configuration")
    try:
        cu.unlock()
    except UnlockError as err:
        print ("Unable to unlock configuration: {0}".format(err))
    dev.close()


In [49]:
print(dev.display_xml_rpc('show route', format='text'))
print(dev.display_xml_rpc('show version', format='text'))

<get-route-information>
</get-route-information>

<get-software-information>
</get-software-information>



In [92]:
#invoke the RPC equivalent to "show route"
rt = dev.rpc.get_route_information()
pprint(etree.tostring(rt, encoding='unicode', pretty_print=True))

#invoke the RPC equivalent to "show version"
sw = dev.rpc.get_software_information()
print(etree.tostring(sw, encoding='unicode',  pretty_print=True))

('<route-information>\n'
 '<!-- keepalive -->\n'
 '<route-table>\n'
 '<table-name>inet.0</table-name>\n'
 '<destination-count>11</destination-count>\n'
 '<total-route-count>11</total-route-count>\n'
 '<active-route-count>11</active-route-count>\n'
 '<holddown-route-count>0</holddown-route-count>\n'
 '<hidden-route-count>0</hidden-route-count>\n'
 '<rt style="brief">\n'
 '<rt-destination>0.0.0.0/0</rt-destination>\n'
 '<rt-entry>\n'
 '<active-tag>*</active-tag>\n'
 '<current-active/>\n'
 '<last-active/>\n'
 '<protocol-name>Static</protocol-name>\n'
 '<preference>5</preference>\n'
 '<age seconds="40598">11:16:38</age>\n'
 '<nh>\n'
 '<selected-next-hop/>\n'
 '<to>172.17.0.1</to>\n'
 '<via>fxp0.0</via>\n'
 '</nh>\n'
 '</rt-entry>\n'
 '</rt>\n'
 '<rt style="brief">\n'
 '<rt-destination>1.1.1.1/32</rt-destination>\n'
 '<rt-entry>\n'
 '<active-tag>*</active-tag>\n'
 '<current-active/>\n'
 '<last-active/>\n'
 '<protocol-name>Direct</protocol-name>\n'
 '<preference>0</preference>\n'
 '<age seco

In [95]:
hostname = sw.xpath("host-name")[0].text
ver = sw.xpath("junos-version")[0].text
print("%s is with software release - %s" % (hostname, ver))

print('\nall routes in inet0')
all_routes = rt.xpath("./route-table/rt")

for r in rt.xpath("./route-table/rt"):
    print(r.findtext('./rt-destination'), r.findtext('./rt-entry/nh/to'))

core1 is with software release - 15.1F5.15

all routes in inet0
0.0.0.0/0 172.17.0.1
1.1.1.1/32 None
2.2.2.2/32 192.168.12.2
172.17.0.0/24 None
172.17.0.101/32 None
192.168.12.0/24 None
192.168.12.1/32 None
222.2.2.0/24 None
222.2.2.2/32 None
224.0.0.2/32 None
224.0.0.5/32 None
2.2.2.2/32 192.168.12.2
0 None
0(S=0) None
1 None
2 None
2(S=0) None
13 None
299808 192.168.12.2
299808(S=0) 192.168.12.2


In [13]:
import jinja2, yaml
templateLoader = jinja2.FileSystemLoader(searchpath=".")
templateEnv = jinja2.Environment(loader=templateLoader, autoescape=False, trim_blocks=False)

config_vars = {
          'interfaces' : [
             { 'physical_interface' : 'ge-0/0/4', 'description' : 'this is ge-0/0/4', 'ip_address' : '10.10.10.1/24' } ,
             { 'physical_interface' : 'ge-0/0/5', 'description' : 'this is ge-0/0/5', 'ip_address' : '10.10.30.1/24' }
           ]
}

TEMPLATE_FILE = 'interface.j2'
template = templateEnv.get_template(TEMPLATE_FILE)
outputText = template.render(config_vars)
print(outputText)



interfaces {

    ge-0/0/4 {
        description "this is ge-0/0/4";
        unit 0 {

            family inet {
                address "10.10.10.1/24";
        }

        }
    } 
    ge-0/0/5 {
        description "this is ge-0/0/5";
        unit 0 {

            family inet {
                address "10.10.30.1/24";
        }

        }
    } 
}


In [14]:
# with Config(dev, mode='private') as cu:
with Config(dev) as cu: 
    cu.load(outputText, format="text", merge=True)
    cu.pdiff()

#     cu.commit(comment='rendered configuration')


[edit interfaces]
+   ge-0/0/4 {
+       description "this is ge-0/0/4";
+       unit 0 {
+           family inet {
+               address 10.10.10.1/24;
+           }
+       }
+   }
+   ge-0/0/5 {
+       description "this is ge-0/0/5";
+       unit 0 {
+           family inet {
+               address 10.10.30.1/24;
+           }
+       }
+   }



In [15]:
    cu.rollback()

True

In [176]:
dev.close()